<a href="https://colab.research.google.com/github/rexbrandy/Neural_Networks/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install 'portalocker>=2.0.0'

In [ ]:
!python -m spacy download en_core_web_sm

In [4]:
import torch
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

import random

SEED = 1111

train_iter, test_iter = AG_NEWS(split=('train','test'))

print(type(train_iter))

for label, line in train_iter:
    print(label, line)
    break

tokenizer = get_tokenizer('basic_english')

<class 'torch.utils.data.datapipes.iter.sharding.ShardingFilterIterDataPipe'>
1 I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was conside